In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/work'

In [ ]:
!kaggle competitions download -c rsna-pneumonia-detection-challenge

100%|███████████████████████████████████████| 3.66G/3.66G [00:39<00:00, 181MB/s]
100%|██████████████████████████████████████| 3.66G/3.66G [00:43<00:00, 90.6MB/s]


In [ ]:
import zipfile as zp

# specify the path and name of the zip file
zip_file = zp.ZipFile('/work/rsna-pneumonia-detection-challenge.zip')

# specify the directory where you want to extract the files
extract_dir = '/work'

# extract all the files from the zip file to the specified directory
zip_file.extractall(extract_dir)

# close the zip file
zip_file.close()

In [15]:
arr = os.listdir('/work/stage_2_train_images')
print(len(arr))

7134


In [17]:
from PIL import Image
import matplotlib.pyplot as plt
import os, sys
import pandas as pd
import numpy as np
import math

normal = '/datasets/acm-drive-data/ACM DATA/normal'
pneumonia = pd.read_csv('/datasets/acm-drive-data/ACM DATA/yolo/pneumonia/pneumonia.csv')
pneumonia_image = '/work/stage_2_train_images'
covid = pd.read_csv('/datasets/acm-drive-data/ACM DATA/yolo/train_image_level.csv')
covid_study = pd.read_csv('/datasets/acm-drive-data/ACM DATA/yolo/train_study_level.csv')
covid_pneumonia_image = '/datasets/acm-drive-data/ACM DATA/yolo/covid_pneumonia'
cardiomegaly = pd.read_csv('/datasets/acm-drive-data/ACM DATA/yolo/Cardiomegaly/BBox_List_2017.csv')
cardiomegaly = cardiomegaly[cardiomegaly['Finding Label'] == 'Cardiomegaly']


pneumonia = pneumonia[pneumonia['Target'] == 1]

arr = covid_study.values
for i in range(len(arr)):
    temp = str(arr[i][0])
    arr[i][0] = temp[:-6]

covid_study = pd.DataFrame({'StudyInstanceUID':arr[:,0],
                           'Negative For Pneumonia':arr[:,1],
                           'Typical Appearance':arr[:,2],
                           'Intermediate Appearance':arr[:,3],
                           'Atypical Appearance':arr[:,4]})

covid = pd.merge(covid, covid_study, on='StudyInstanceUID', how='inner')
covid = covid[covid['Typical Appearance'] == 1]
covid = covid.dropna(axis=0)


pneumonia

,patientId,x,y,width,height,Target
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1
14,00aecb01-a116-45a2-956c-08d2fa55433f,288.0,322.0,94.0,135.0,1
...,...,...,...,...,...,...
30220,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,316.0,504.0,179.0,273.0,1
30221,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,609.0,464.0,240.0,284.0,1
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1


In [ ]:
import pandas as pd
#pneumonia = pd.read_csv('/datasets/acm-drive-data/ACM DATA/yolo/pneumonia/pneumonia.csv')
#pneumonia

covid = pd.read_csv('/datasets/acm-drive-data/ACM DATA/yolo/train_image_level.csv')
covid_study = pd.read_csv('/datasets/acm-drive-data/ACM DATA/yolo/train_study_level.csv')
arr = covid_study.values
for i in range(len(arr)):
    temp = str(arr[i][0])
    arr[i][0] = temp[:-6]

covid_study = pd.DataFrame({'StudyInstanceUID':arr[:,0],
                           'Negative For Pneumonia':arr[:,1],
                           'Typical Appearance':arr[:,2],
                           'Intermediate Appearance':arr[:,3],
                           'Atypical Appearance':arr[:,4]})

covid = pd.merge(covid, covid_study, on='StudyInstanceUID', how='inner')
covid = covid.dropna(axis=0)
covid

,id,boxes,label,StudyInstanceUID,Negative For Pneumonia,Typical Appearance,Intermediate Appearance,Atypical Appearance
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0,1,0,0
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,0,0,0,1
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,0,1,0,0
5,0022227f5adf_image,"[{'x': 1857.2065, 'y': 508.30565, 'width': 376...",opacity 1 1857.2065 508.30565 2233.23384 907.8...,84543edc24c2,0,0,1,0
...,...,...,...,...,...,...,...,...
6327,ffbeafe30b77_image,"[{'x': 1472.09879, 'y': 1182.64415, 'width': 3...",opacity 1 1472.09879 1182.64415 1773.17398 146...,f75774536369,0,1,0,0
6328,ffcc16bbf428_image,"[{'x': 2529.03977, 'y': 347.67053, 'width': 58...",opacity 1 2529.03977 347.67053 3113.65526 1484...,5d786634f354,0,1,0,0
6331,ffd9b6cf2961_image,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,0,1,0,0
6332,ffdc682f7680_image,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,0,1,0,0


In [ ]:
import os
bBoxes = []
inputValues = []

count_normal = 0
string = '/datasets/acm-drive-data/ACM DATA/normal/'
for v in os.listdir(string):
    path = string + v
    inputValues.append(path)
    bBoxes.append(None)
    count_normal += 1

count_pneumonia = 0
pneumonia_arr = pneumonia.values
string = '/datasets/acm-drive-data/ACM DATA/yolo/pneumonia/pneumonia_images/'
for v in os.listdir(string):
    idName = v[:-4]
    if idName in pneumonia_arr[:,0]:
        path = string + v
        inputValues.append(path)
        count_pneumonia += 1
        df_rows = pneumonia.loc[pneumonia['patientId']==idName]
        rows = df_rows.to_numpy()

        numBounding = []
        for i in range(len(rows)):
            box = {}
            box['x'] = rows[i][1]
            box['y'] = rows[i][2]
            box['width'] = rows[i][3]
            box['height'] = rows[i][4]
            numBounding.append(box)
        bBoxes.append(numBounding)


count_covid = 0
covid_arr = covid.values
string = '/datasets/acm-drive-data/ACM DATA/yolo/covid_pneumonia/'
for v in os.listdir(string):
    idName = v[:-4] + '_image'
    numBounding = []
    if idName in covid_arr[:,0]:
        path = string + v 
        df_rows = covid.loc[covid['id']==idName]
        rows = df_rows.to_numpy()
        inputValues.append(path)
        count_covid += 1
        bBoxes.append(rows[0][1])

In [ ]:
count_cardio = 0
cardiomegaly_arr = cardiomegaly.values
string = '/datasets/acm-drive-data/ACM DATA/Part8'
for v in os.listdir(string):
    if v in cardiomegaly_arr[:,0]:
        temp = string + '/' + str(v)
        df_row = cardiomegaly.loc[cardiomegaly['Image Index']==v]
        rows = df_row.to_numpy()
        inputValues.append(temp)
        count_cardio += 1
        numBounding = []
        for i in range(len(rows)):
            box = {}
            box['x'] = rows[i][2]
            box['y'] = rows[i][3]
            box['width'] = rows[i][4]
            box['height'] = rows[i][5]
            numBounding.append(box)
        bBoxes.append(numBounding)

In [ ]:
j = 1
for i in range(1, 4):
    if j == 8:
        j += 1
    while True:
        if i == 1 and j == 5 or i == 2 and j == 8 or i == 3 and j == 13:
            break
        #print(i, j)
        cardio = '/datasets/cardiomegaly' + str(i) + '/Part-' + str(j)
        cardio_dir = os.listdir(cardio)
        for v in cardio_dir:
            if v in cardiomegaly_arr[:,0]:
                temp = string + '/' + str(v)
                df_row = cardiomegaly.loc[cardiomegaly['Image Index']==v]
                rows = df_row.to_numpy()
                print(rows)
                inputValues.append(temp)
                count_cardio += 1
                numBounding = []
                for k in range(len(rows)):
                    box = {}
                    box['x'] = rows[k][2]
                    box['y'] = rows[k][3]
                    box['width'] = rows[k][4]
                    box['height'] = rows[k][5]
                    numBounding.append(box)
                bBoxes.append(numBounding)
        j += 1

[['00000732_005.png' 'Cardiomegaly' 427.932203389831 464.0
  412.203389830508 344.949152542373 nan nan nan]]
[['00000756_001.png' 'Cardiomegaly' 289.320634920635 325.079365079365
  601.396825396825 430.18835978836 nan nan nan]]
[['00000377_004.png' 'Cardiomegaly' 387.796610169492 419.525423728814
  502.237288135593 367.728813559322 nan nan nan]]
[['00000211_041.png' 'Cardiomegaly' 224.0 416.849725238347 512.0
  430.64406779661 nan nan nan]]
[['00000661_000.png' 'Cardiomegaly' 298.847457627119 437.459894729873
  548.881355932203 428.474576271186 nan nan nan]]
[['00000845_000.png' 'Cardiomegaly' 370.590476190476 367.339682539683
  454.027513227513 424.77037037037 nan nan nan]]
[['00000211_019.png' 'Cardiomegaly' 340.067796610169 422.779661016949
  501.152542372881 406.779661016949 nan nan nan]]
[['00000398_003.png' 'Cardiomegaly' 369.506878306878 389.01164021164
  484.368253968254 376.008465608466 nan nan nan]]
[['00000740_000.png' 'Cardiomegaly' 301.240211640212 384.677248677249
  476.7

In [ ]:
outputValues = []
for _ in range(count_normal):
    outputValues.append('Normal')

for _ in range(count_pneumonia):
    outputValues.append('Pneumonia')

for _ in range(count_covid):
    outputValues.append('COVID')

for _ in range(count_cardio):
    outputValues.append('Cardiomegaly')

In [ ]:
print(len(inputValues), len(bBoxes), len(outputValues), count_cardio)

12243 12243 12243 135


In [ ]:
df = pd.DataFrame({'input':inputValues,
                    'box':bBoxes,
                    'output':outputValues})
df.to_csv('object.csv')

In [ ]:
import pandas as pd

df = pd.read_csv('object.csv')
df
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,input,box,output
0,/datasets/acm-drive-data/ACM DATA/normal/NORMA...,NaN,Normal
1,/datasets/acm-drive-data/ACM DATA/normal/NORMA...,NaN,Normal
2,/datasets/acm-drive-data/ACM DATA/normal/NORMA...,NaN,Normal
3,/datasets/acm-drive-data/ACM DATA/normal/NORMA...,NaN,Normal
4,/datasets/acm-drive-data/ACM DATA/normal/NORMA...,NaN,Normal
...,...,...,...
12238,/datasets/acm-drive-data/ACM DATA/Part8/000302...,"[{'x': 312.076190476191, 'y': 339.166137566138...",Cardiomegaly
12239,/datasets/acm-drive-data/ACM DATA/Part8/000285...,"[{'x': 310.779661016949, 'y': 400.0, 'width': ...",Cardiomegaly
12240,/datasets/acm-drive-data/ACM DATA/Part8/000286...,"[{'x': 404.182010582011, 'y': 302.32380952381,...",Cardiomegaly
12241,/datasets/acm-drive-data/ACM DATA/Part8/000302...,"[{'x': 289.320634920635, 'y': 420.435978835979...",Cardiomegaly


In [ ]:
n = df[df['output'] == 'Normal']
p = df[df['output'] == 'Pneumonia']
c = df[df['output'] == 'COVID']
ca = df[df['output'] == 'Cardiomegaly']

In [ ]:
normal, pneumonia, covid, cardio = n.values, p.values, c.values, ca.values

In [ ]:
import numpy as np
import pydicom

def convert(path):
    image = pydicom.dcmread(path)
    image = image.pixel_array.astype(float)
    rescaled = (np.maximum(image, 0)/image.max()) * 255 # Float Pixels
    final_image = np.uint8(rescaled)
    final_image = Image.fromarray(final_image)
    return final_image

In [ ]:
df = pd.read_csv("actualObject.csv", error_bad_lines=False)
df

,Unnamed: 0,Absolute Path,Relative Path,Bounding Boxes,Output
0,0,pneumoniaPics/149d826e-7078-4b2c-888a-d8811c72...,149d826e-7078-4b2c-888a-d8811c72caf8.jpg,"[{'x': 55.25, 'y': 107.0, 'width': 44.75, 'hei...",Pneumonia
1,1,pneumoniaPics/b19a9422-a790-4a43-b59f-65a3bf0f...,b19a9422-a790-4a43-b59f-65a3bf0f16be.jpg,"[{'x': 152.5, 'y': 65.5, 'width': 28.5, 'heigh...",Pneumonia
2,2,pneumoniaPics/3d4dfd10-61c7-4305-816f-fde544d1...,3d4dfd10-61c7-4305-816f-fde544d1f3a3.jpg,"[{'x': 56.5, 'y': 137.5, 'width': 59.5, 'heigh...",Pneumonia
3,3,pneumoniaPics/d4e0b27e-79ad-4af1-bd49-6c7feaf3...,d4e0b27e-79ad-4af1-bd49-6c7feaf3f84c.jpg,"[{'x': 131.5, 'y': 59.0, 'width': 77.75, 'heig...",Pneumonia
4,4,pneumoniaPics/9acc3015-cffb-414a-a8b1-87555ba7...,9acc3015-cffb-414a-a8b1-87555ba73b50.jpg,"[{'x': 141.25, 'y': 113.75, 'width': 72.25, 'h...",Pneumonia
...,...,...,...,...,...
9076,9076,/datasets/acm-drive-data/ACM DATA/Part8/000175...,00017514_008.png,"[{'x': 154.25, 'y': 55.5, 'width': 45.5, 'heig...",Cardiomegaly
9077,9077,/datasets/acm-drive-data/ACM DATA/Part8/000175...,00017524_028.png,"[{'x': 154.25, 'y': 55.5, 'width': 45.5, 'heig...",Cardiomegaly
9078,9078,/datasets/acm-drive-data/ACM DATA/Part8/000178...,00017893_005.png,"[{'x': 154.25, 'y': 55.5, 'width': 45.5, 'heig...",Cardiomegaly
9079,9079,/datasets/acm-drive-data/ACM DATA/Part8/000166...,00016606_000.png,"[{'x': 154.25, 'y': 55.5, 'width': 45.5, 'heig...",Cardiomegaly


In [ ]:
import numpy as np
import pydicom

def convert(path):
    image = pydicom.dcmread(path)
    image = image.pixel_array.astype(float)
    rescaled = (np.maximum(image, 0)/image.max()) * 255 # Float Pixels
    final_image = np.uint8(rescaled)
    final_image = Image.fromarray(final_image)
    return final_image

In [ ]:
from PIL import Image, ImageOps
import os

res = df1.values

absolute = []
relative = []
bBox = []
output = []

for i in range(1802,7814):
    path = convert(res[i][1])
    arrSplit = res[i][1].split('/')
    if not os.path.exists('pneumoniaPics'):
        os.makedirs('pneumoniaPics')
    rel = arrSplit[7][:-4] + '.jpg'
    path.save('pneumoniaPics/' + arrSplit[7][:-4] + '.jpg')

In [ ]:
test_record = dataset.location + '/test/cells.tfrecord'
train_record = dataset.location + '/train/cells.tfrecord'
label_map = dataset.location + '/train/data.pbtxt'

In [ ]:
MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 16
    }
}

In [ ]:
chosen_model = 'efficientdet-d0'

num_steps = 1000
num_eval_steps = 50 #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size']

In [ ]:
%mkdir /content/models/research/deploy/
%cd /content/models/research/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

In [ ]:
%cd /content/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

In [ ]:
#Prepare Data
pipeline_fname = '/content/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

In [ ]:
# Reference For Code Below: https://www.tensorflow.org/api_docs

import re

%cd /content/models/research/deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

In [ ]:
pipeline_file = '/content/models/research/deploy/pipeline_file.config'
model_dir = '/content/training/'

In [ ]:
# Reference For Code Below: https://www.tensorflow.org/api_docs

!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

In [ ]:
# Show Contents 
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=40fc931a-05f8-4cdf-a2a6-9f4c4789f625' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>